# Advanced Example: Reward-Based CTMC Analysis

This example demonstrates the `setReward` feature for defining custom reward functions on a queueing network model and computing steady-state expected rewards using the CTMC solver.

The model is a simple M/M/1/K queue with finite buffer, and we define several reward functions:
- Queue length
- Utilization
- Blocking probability
- Quadratic queue cost

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun createRewardModel(): Network {
    val model = Network("RewardExample")
    
    // Block 1: nodes
    val source = Source(model, "Source")
    val queue = Queue(model, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model, "Sink")
    
    // Set finite buffer capacity
    queue.setNumberOfServers(1)
    queue.setCap(10)  // Maximum 10 jobs in the system
    
    // Block 2: job classes
    val oclass = OpenClass(model, "Class1")
    source.setArrival(oclass, Exp(2.0))  // Arrival rate = 2
    queue.setService(oclass, Exp(3.0))   // Service rate = 3 (utilization ~ 0.67)
    
    // Block 3: topology
    model.addLink(source, queue)
    model.addLink(queue, sink)
    
    // Define Reward Functions
    // Reward 1: Queue length (number of jobs in the queue)
    model.setReward("QueueLength") { state, sn -> state[1].toDouble() }
    
    // Reward 2: Utilization (1 if server busy, 0 if idle)
    model.setReward("Utilization") { state, sn -> minOf(state[1], 1).toDouble() }
    
    // Reward 3: Blocking indicator (1 if buffer full, 0 otherwise)
    model.setReward("BlockingProb") { state, sn -> if (state[1] >= 10) 1.0 else 0.0 }
    
    // Reward 4: Weighted queue cost (quadratic penalty for long queues)
    model.setReward("QueueCost") { state, sn -> (state[1] * state[1]).toDouble() }
    
    return model
}

// Create the model
val model = createRewardModel()

## About Reward Functions

Reward functions allow computing custom metrics from the underlying Markov chain:

- **State-based**: Rewards depend on the system state (queue lengths, etc.)
- **Steady-state average**: E[R] = Σᵢ πᵢ × r(sᵢ) where πᵢ is steady-state probability
- **Flexible**: Can compute any function of the state

For M/M/1/K queue with λ=2, μ=3, K=10:
- ρ = λ/μ = 2/3 ≈ 0.67
- Expected queue length and other metrics computed via CTMC analysis

In [ ]:
// Solve with CTMC Solver
println("Solving with CTMC solver...\n")

val solver = CTMC(model)

// Get Steady-State Expected Rewards
val rewards = solver.getAvgReward()

println("\n=== Steady-State Expected Rewards ===")
rewards.forEach { (name, value) ->
    println(String.format("%15s: %.6f", name, value))
}